In [ ]:
# Initializing
from google.colab import drive
import re
import os
import math
import keras
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pathlib
from keras.utils import np_utils
import cv2

drive.mount('/content/drive')
# Define constant value
RESEARCH_WORK_PATH = "/content/drive/My Drive/Colab Notebooks/BachelorResearch/"
MODEL_PATH = RESEARCH_WORK_PATH + "models/"
DATA_PATH = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/unknown_img/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def path_to_pic(dir_folder):
    list_of_pic = []
    for file in os.listdir(dir_folder):
        if file.endswith(".jpg"):
            directory = "%s%s" % (dir_folder, file)
            list_of_pic.append(directory)
    return list_of_pic

def make_unknown_dataset(path):
  x = []
  y = []
  for pic in path_to_pic(path):
    im = np.array(Image.open(pic))
    x.append(im)
    y.append(int(pathlib.Path(pic).stem[1])-1)
  x = np.array(x)
  x = x.astype("float32")
  x /= 255
  y = np.array(y)
  return x,y

In [ ]:
x_raw ,y = make_unknown_dataset(DATA_PATH)
y = np_utils.to_categorical(y, 4)
print("[*INFO] Dataset bias:", y.sum(axis=0))
for m in os.listdir(MODEL_PATH):
  print("[MODEL]", pathlib.Path(m).name)
  try:
    model = keras.models.load_model(MODEL_PATH + m)
  except OSError:
    print("[ERROR]This model seems broken.... (> <). Pass this Evaluation.")
    break
  s = model.input.shape
  print("input shape:", s)
  x = x_raw[:,:s[1],:]
  try:
    x = x.reshape(x.shape[0], s[1], s[2], s[3])
  except ValueError:
    x_rgb = []
    for i in x:
      tmp = cv2.cvtColor(i,cv2.COLOR_GRAY2RGB)
      # tmp = cv2.resize(tmp,(s[1],s[2]))
      x_rgb.append(tmp)
    x = np.array(x_rgb)
    x = x.reshape(x.shape[0], s[1], s[2], s[3])
    
  score = model.evaluate(x, y, verbose=2)

  print('accuracy:', score[1])
  print('loss:', score[0])
  print()


[330. 349. 345. 346.]
[MODEL] efficient_net_v2_vanilla_crop02
input shape: (None, 102, 173, 3)
43/43 - 3s - loss: 1.1189 - accuracy: 0.5482 - 3s/epoch - 75ms/step
accuracy: 0.5481751561164856
loss: 1.1189429759979248

[MODEL] efficient_net_v2_vanilla_crop_01
input shape: (None, 224, 173, 3)
43/43 - 4s - loss: 1.1420 - accuracy: 0.5628 - 4s/epoch - 87ms/step
accuracy: 0.5627737045288086
loss: 1.1420042514801025

[MODEL] efficient_net_v2_vanilla_crop015
input shape: (None, 150, 173, 3)
43/43 - 3s - loss: 1.0720 - accuracy: 0.5555 - 3s/epoch - 79ms/step
accuracy: 0.5554744601249695
loss: 1.0719879865646362

[MODEL] mnist_crop0.1
input shape: (None, 224, 173, 1)
43/43 - 1s - loss: 1.3863 - accuracy: 0.2547 - 949ms/epoch - 22ms/step
accuracy: 0.2547445297241211
loss: 1.3863213062286377

[MODEL] mnist_crop0.15
input shape: (None, 150, 173, 1)
43/43 - 1s - loss: 1.3864 - accuracy: 0.2547 - 749ms/epoch - 17ms/step
accuracy: 0.2547445297241211
loss: 1.3863636255264282

[MODEL] mnist_crop0.2
inp